Installation of libraries

In [ ]:
!pip install rasterio
!pip install numpy
!pip install pandas
!pip install pyproj

In [26]:
import rasterio 
import pandas as pd
import numpy as np
import os
from math import radians, sin, cos, acos
import pyproj

Functions to read and process map

In [27]:
#function to read coordinates in CRS and pixel intensity values and create a data frame of the same
def read_map(path_map):
  with rasterio.open(path_map) as dataset: 
      val = dataset.read(1) 
      no_data=dataset.nodata
      data = [(dataset.xy(x,y)[0],dataset.xy(x,y)[1],val[x,y]) for x,y in np.ndindex(val.shape) if val[x,y] != no_data]
      est = [i[0] for i in data]
      nor = [i[1] for i in data]
      pix = [i[2] for i in data]
      conc = pd.DataFrame({"easting":est,'northing':nor,"pixel":pix})
  return conc
    

In [28]:
#function to convert CRS values into latitude and longitude
def to_lat_long(e,n):
  lat=[]
  long=[]
  x1=e
  y1=n
  x1 = np.array(x1, int)
  x1=x1.tolist()
  length = len(x1)   
  #print(length)
  wgs84 = pyproj.Proj(projparams = 'epsg:4326')
  InputGrid = pyproj.Proj(projparams = 'epsg:3976')
  for i in range(length):
    x2,y2=pyproj.transform(InputGrid,wgs84 , x1[i], y1[i])
    lat.append(x2)
    long.append(y2)
  return lat,long

In [29]:
#passes dataframe to the function "to_lat_long" to get latitude and longitude values
def convert(df):
  conc = df
  laty=[]
  longy=[]
  laty,longy=to_lat_long(conc['easting'],conc['northing'])
  conc['Latitude'] =laty
  conc['Longitude']=longy
  return conc

In [30]:
#function to obtain the coordinates of sea ice concentration greater than 20% and that of the coastline alog every longitude
def slice_n_dice(df):
  conc=df
  conc = conc.astype({'Longitude':'int'})
  conc=conc.sort_values(by=['Longitude','pixel']).set_index(['Longitude', 'pixel'])
  conc = conc.loc[(conc.index.get_level_values('pixel')>=200)]
  conc=conc.reset_index()

  concy1 = conc.groupby('Longitude').first()
  concy1.rename(columns = {'pixel':'Sea_Ice_Conc','Latitude':'Sea_Ice_Lat','easting':'Sea_Ice_Est','northing':'Sea_Ice_Nth'}, inplace = True)

  concy2 = conc.sort_values(by=['Longitude','Latitude'],ascending=[True, True])
  concy2 = concy2[(concy2['pixel']==2540)]

  concy3 = concy2.groupby('Longitude').last()

  concy5 = pd.concat([concy1,concy3],axis=1)
  long=concy5.index.get_level_values('Longitude')
  concy5["Long"]=long

  return concy5

In [31]:
#function to calculate distance
def dist(cols):
    long=cols[0]
    sil=cols[1]
    coast1=cols[2]
        
    slat = radians(sil)
    slon = radians(long)
    elat = radians(coast1)
    elon = radians(long)

    dist = 6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon))
    return dist
        

In [32]:
#function to calculate sea ice concentration in %
def sic(cols):
    sic=cols
    if sic>2500:
        sic=0

    else:
        sic=sic/10
    return  sic

In [33]:
#function to limit analysis only to Antarctic continent
def sea_ice_edge(df):
  concy5 = df
  concy5['Distance']=concy5[['Long','Sea_Ice_Lat','Latitude']].apply(dist,axis=1)
  concy5.drop("Long",axis=1,inplace=True)

  concy5["Sea_Ice_Conc(%)"]= concy5["Sea_Ice_Conc"].apply(sic)
  concy5.rename(columns = {'Sea_Ice_Conc':'Sea_Ice_Pixel'}, inplace = True)
  concy5.drop(concy5[concy5['Sea_Ice_Conc(%)'] == 0].index, inplace = True)
  concy5 = concy5[concy5['Latitude']<=-55]

  return concy5

In [34]:
#function to write to excel
def write(df,path_sheet,path_map,sn):
  concy5 = df
  concy5.to_excel(path_sheet, sheet_name = sn)
  !rm -rf "$path_map"

Updating urls and file paths

In [35]:
#function to update the map url
def update_map(link,j):
  map = link
  map= map[:89] + j + map[93:]
  #print(map)
  test = os.system('wget %s' %map)
  return test


In [36]:
#function to update the file path of the map
def update_path_map(path,j):
  path= path[:11] + j + path[15:]
  j = str(int(j)+1)
  return path

In [37]:
#function to update the file path of the excel sheet
def update_path_sheet(path,j):
  length = len(path)
  length = length - 9
  end = length + 4
  path= path[:length] + j + path[end:]
  j = str(int(j)+1)
  return path


Heart of the program

In [38]:
#to acsses google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
#set the url of first map, file path of first map and excel sheet of first map as constants to enable updating as program proceeds
j='1979'
link = 'https://masie_web.apps.nsidc.org/pub/DATASETS/NOAA/G02135/south/monthly/geotiff/01_Jan/S_199301_concentration_v3.0.tif'
path_map = '/content/S_199301_concentration_v3.0.tif'
path_sheet = '/content/drive/MyDrive/NCPOR/RESULTS/January/1993.xlsx'

In [40]:
#for loop to run through 45 years for every month
for i in range(45): 
  print("Processing year ",j)

  flag = update_map(link,j)
  path_map = update_path_map(path_map,j)
  path_sheet = update_path_sheet(path_sheet,j)
  j = str(int(j)+1)

  if flag == 0:
    conc = read_map(path_map)
    conc = convert(conc)
    concy5 = slice_n_dice(conc)
    concy5 = sea_ice_edge(concy5)

    sn = str(int(j) - 1)
    write(concy5,path_sheet,path_map,sn)

    print("Finished Processing year ",sn)
    print(" ")
  else:
    print("Map not found..moving to next year.")

  if j == 2022:
    break

print("DONE")

Processing year  1993


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  


Finished Processing year  1993
 
Processing year  1994


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  


Finished Processing year  1994
 
Processing year  1995


...

Finished Processing year  2020
 
Processing year  2021


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  


Finished Processing year  2021
 
Processing year  2022


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  


Finished Processing year  2022
 
Processing year  2023
Map not found..moving to next year.
Processing year  2024
Map not found..moving to next year.
Processing year  2025
Map not found..moving to next year.
Processing year  2026
Map not found..moving to next year.
Processing year  2027
Map not found..moving to next year.
Processing year  2028
Map not found..moving to next year.
Processing year  2029
Map not found..moving to next year.
Processing year  2030
Map not found..moving to next year.
Processing year  2031
Map not found..moving to next year.
Processing year  2032
Map not found..moving to next year.
Processing year  2033
Map not found..moving to next year.
Processing year  2034
Map not found..moving to next year.
Processing year  2035
Map not found..moving to next year.
Processing year  2036
Map not found..moving to next year.
Processing year  2037
Map not found..moving to next year.
